In [25]:
# From https://docs.opencv.org/3.1.0/d4/dc6/tutorial_py_template_matching.html
# And https://www.geeksforgeeks.org/template-matching-using-opencv-in-python/
%matplotlib inline
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
import random
import imutils

from IPython.display import display, HTML
import cv2
import base64


def imshow(name, imageArray):
     _, png = cv2.imencode('.png', imageArray)
     _encoded = base64.b64encode(png)
     return HTML(data='''<img alt="{0}" src="data:image/png;base64, {1}"/>'''.format(name, _encoded.decode('ascii')))


OPENED_COLUMNS = {}

def check_file_for_template(image_name, template, method=cv2.TM_CCOEFF, show = False, scale = None,
                           use_global=False):
    if use_global:
        if image_name not in OPENED_COLUMNS:
            OPENED_COLUMNS[image_name] = cv2.imread(image_name, 0).copy()
        img = OPENED_COLUMNS[image_name]
    else: 
        ground = cv2.imread(image_name, 0)
        img = ground.copy()
    # Template size
    w, h = template.shape[::-1]
    w2, h2 = img.shape[::-1]
    
    # If the template is bigger
    if w > w2 or h > h2:
        return -1, None, None
    
    if scale:
        # resize the image according to the scale, and keep track
        # of the ratio of the resizing
        resized = imutils.resize(img, width = int(img.shape[1] * scale))
        r = img.shape[1] / float(resized.shape[1])

        # if the resized image is smaller than the template, then break
        # from the loop
        if resized.shape[0] < h or resized.shape[1] < w:
            return -1, None, None
            
        img = resized
    
    # Apply template Matching
    res = cv2.matchTemplate(img, template, method)
    
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    cv2.rectangle(img, top_left, bottom_right, (0,0,255), 1)

    if show:
        cv2.imshow("Image", img)
        
    return max_val, top_left, bottom_right
    
def find_line_best_match(subimage, folder):
    # All the 6 methods for comparison in a list
    template = cv2.imread(subimage, 0)
    
    best_match = (0, None)
    for filename in glob.glob(os.path.join(folder, "*.pseg.png")):
        max_val, top_left, bottom_right = check_file_for_template(filename, template)
        if max_val > best_match[0]:
            best_match = (max_val, filename)

    print("Best match is " + best_match[1])
    return best_match[1]


def find_column_match(image_set, folder):
    images = glob.glob(os.path.join(image_set, "*.png"))
    random.shuffle(images)
    column = [
        find_line_best_match(image, folder)
        for image in images[:3]
    ]
    
    if len(set(column)) == 1:
        return column[0]
    else:
        return ", ".join(column)

def find_best_scale(subimage, column_matcher):
    template = cv2.imread(subimage, 0)
    
    best_match = (0, 1, 0, 0)
    for scale in np.linspace(0.2, 1.0, 20)[::-1]:
        max_val, top_left, bottom_right = check_file_for_template(
            image_name=column_matcher,
            template=template,
            scale=scale
        )
        if max_val == -1:
            break
        if max_val > best_match[0]:
            best_match = (max_val, scale, top_left, bottom_right)

    print("Best match is {}".format(best_match[1]))
    return best_match

In [44]:
def tuple_scale(tup, scale):
    return tuple([int(x/scale) for x in tup])

def show_best_scale(subimage, column_matcher):
    score, scale, top, bottom = find_best_scale(subimage, column_matcher)
    
    print("Found it ! : {}".format(scale))
    
    # draw a bounding box around the detected region
    column = cv2.imread(column_matcher, 0).copy()
    template = cv2.imread(subimage, 0)
    tW, tH = template.shape[::-1]
    print(top)
    print(bottom)
    print(tuple_scale(top, scale))
    print(tuple_scale(bottom, scale))
    cv2.rectangle(
        column,
        tuple_scale(top, scale),
        tuple_scale(bottom, scale),
        (0, 0, 255),
        2
    )
    
    imshow("Hello", column)
    
    return scale, imshow("Hello", column)

scale, img = show_best_scale(
    "/home/thibault/dev/ocropus-to-tf-crnn/input/test/saintmartin/0003/010013.bin.png",
    "columns/0396.bin.png"
)
img

Best match is 0.7473684210526317
Found it ! : 0.7473684210526317
(40, 589)
(630, 635)
(53, 788)
(842, 849)


In [ ]:
# Get the best match from the original Ocropus

data = {}

for column in glob.glob("/home/thibault/dev/ocropus-to-tf-crnn/input/**/**/**"):
    print("Searching for " + column)
    result = find_column_match(column, "columns")
    print("Result : " + str(resullt))
    data[column] = result
    
print(data)
import json
print(json.dumps(data))